<h3><center>Applied Data Mining</center></h3>
<h1><center>Bayesian Network</h1>

### Before you arrive at the lab:
You should read through the entire document and look up anything you don’t quite remember from the lectures.

### After the lab:
Within two weeks after this lab session you must hand in your report. Your report should explain your results, and how you achieved them. Make sure you include any relevant information to your explanation.

This lab is designed to introduce you the use of Bayesian Network (BN). By the end of this lab session you will:

- Learn how to estimate BN parameters from data
- Learn how to use BN for reasoning and decision making

For this lab, you will use two datasets to construct BN:

- Artificial Smart Grid Data
- Real Smart Grid Data

The records of the failures of smart grids are shown in the two data sets.

- data_artificial.xlsx: an artificial failure dataset, containing 700 observations.
- data_real.xlsx: a real historical failure dataset from a smart grid in Sweden, containing 1657 observations.

#### data_artificial.xlsx

The attributes in this dataset are: Number_of_Customers, Time, Day, Season, Weather, Demand_Factor, Overload, and Outage_Duration, with values listed in the following.

- Season: Spring, Summer, Autumn, Winter
- Outage_Duration: Less_than_1H, More_than_1H
- Number_of_Customers: Low, High
- Overload: Yes, No
- Weather: Cold, Warm
- Time: Morning, Afternoon, Evening, Night
- Demand_Factor: Low, Medium, High
- Day: Weekdays, Weekend

#### data_real.xlsx

The attributes in this dataset are: Year, Season, Hour, Outage duration, Switchgear, Volt.level, Cause, Facility part, with values listed in the following.

- Year: before_2011, after_2011
- Season: Spring, Summer, Autumn, Winter
- Hour: Morning, Afternoon, Evening, Night
- Outage_Duration: Otg<=1, 1<Otg<=2, Otg>2
- Switchgear: H2, H3, H4, H7, H8, H10, LINEHED EON
- Volt.level: 0.2<= u <=1.0, 10<= u <=12
- Cause: Digging, Fabrication fault, and …
- Facility part: Ground cable pillar, Concr.sec.substation indoor man., and …

Now, let's first run the following codes to explore the data and the specific variables of each files.

In [6]:
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_excel(r"lab2_data/Data_artificial.xlsx")
data.head()

,Season,Outage_Duration,Number_of_Customers,Overload,Weather,Time,Demand_Factor,Day
0,Autumn,Less_than_1H,Low,Yes,Cold,Morning,Low,Weekdays
1,Winter,Less_than_1H,Low,No,Cold,Evening,Low,Weekdays
2,Spring,More_than_1H,Low,No,Cold,Evening,Low,Weekdays
3,Winter,Less_than_1H,High,No,Warm,Morning,Low,Weekdays
4,Spring,More_than_1H,Low,No,Cold,Morning,Low,Weekend


In [3]:
from collections import Counter
# Season
Counted0 = Counter(data.Season)
print('Season')
print('------')
print('Autumn: ',(Counted0['Autumn']/len(data))*100,'Winter: ',(Counted0['Winter']/len(data))*100,'Spring: ',(Counted0['Spring']/len(data))*100, 'Summer: ',(Counted0['Summer']/len(data))*100)

# Outage_Duration
Counted1 = Counter(data.Outage_Duration)
print('Outage Duration')
print('---------------')
print('Less than 1 H: ',(Counted1['Less_than_1H']/len(data))*100,'More than 1 H: ',(Counted1['More_than_1H']/len(data))*100)

# Number_of_Customers
Counted2 = Counter(data.Number_of_Customers)
print('Number of Customers')
print('--------------------')
print('Low: ',(Counted2['Low']/len(data))*100,'High: ',(Counted2['High']/len(data))*100)

# Overload
Counted3 = Counter(data.Overload)
print('Overload')
print('--------')
print('No: ',(Counted3['No']/len(data))*100,'Yes: ',(Counted3['Yes']/len(data))*100)

# Weather
Counted4 = Counter(data.Weather)
print('Weather')
print('-------')
print('Cold: ',(Counted4[' Cold']/len(data))*100,'Warm: ',(Counted4[' Warm']/len(data))*100)

# Time
Counted5 = Counter(data.Time)
print('Time')
print('----')
print('Morning: ',(Counted5['Morning']/len(data))*100,'Afternoon: ',(Counted5['Afternoon']/len(data))*100,'Evening: ',(Counted5['Evening']/len(data))*100,'Night: ',(Counted5['Night']/len(data))*100)


# Demand_Factor
Counted6 = Counter(data.Demand_Factor)
print('Demand Factor')
print('-------------')
print('Low: ',(Counted6['Low']/len(data))*100,'Medium: ',(Counted6['Medium']/len(data))*100,'High: ',(Counted6['High']/len(data))*100)

# Day
Counted7 = Counter(data.Day)
print('Day')
print('---')
print('Weekdays: ',(Counted7['Weekdays']/len(data))*100,'Weekend: ',(Counted7['Weekend']/len(data))*100)

Season
------
Autumn:  25.857142857142858 Winter:  24.857142857142858 Spring:  24.142857142857142 Summer:  25.142857142857146
Outage Duration
---------------
Less than 1 H:  45.285714285714285 More than 1 H:  54.714285714285715
Number of Customers
--------------------
Low:  59.42857142857143 High:  40.57142857142857
Overload
--------
No:  64.42857142857143 Yes:  35.57142857142857
Weather
-------
Cold:  60.857142857142854 Warm:  39.14285714285714
Time
----
Morning:  31.71428571428571 Afternoon:  29.28571428571429 Evening:  29.28571428571429 Night:  9.714285714285714
Demand Factor
-------------
Low:  53.0 Medium:  23.142857142857142 High:  23.857142857142858
Day
---
Weekdays:  60.57142857142858 Weekend:  39.42857142857143


In [7]:
# 2nd data set
data = pd.read_excel("lab2_data/Data_real.xlsx")
data.head()

,Year,Season,Hour,Outage_Duration,Switch_Gear,Volt_Level,Cause,Facility_Part
0,after_2011,Spring,Morning,1<Otg<=2,H7,0.2 <= u <= 1.0,Digging,Concr.sec.substation indoor man.
1,before_2011,Summer,Morning,1<Otg<=2,H10,0.2 <= u <= 1.0,Fabrication fault,Concr.sec.substation indoor man.
2,before_2011,Summer,Evening,1<Otg<=2,H10,0.2 <= u <= 1.0,Fabrication fault,Concr.sec.substation indoor man.
3,before_2011,Autumn,Afternoon,1<Otg<=2,H10,0.2 <= u <= 1.0,Fuse break,Concr.sec.substation indoor man.
4,before_2011,Spring,Morning,1<Otg<=2,H3,0.2 <= u <= 1.0,Fuse break,Concr.sec.substation indoor man.


In [8]:
from collections import Counter
# Year
Counted0 = Counter(data.Year)
print('Year')
print('------')
print('After 2011: ',(Counted0['after_2011']/len(data))*100,'Before 2011: ',(Counted0['before_2011']/len(data))*100)


# Season
Counted0 = Counter(data.Season)
print('Season')
print('------')
print('Autumn: ',(Counted0['Autumn']/len(data))*100,'Winter: ',
      (Counted0['Winter']/len(data))*100,'Spring: ',(Counted0['Spring']/len(data))*100, 'Summer: ',
      (Counted0['Summer']/len(data))*100)

# Hour
Counted5 = Counter(data.Hour)
print('Hour')
print('----')
print('Morning: ',(Counted5['Morning']/len(data))*100,'Afternoon: ',
      (Counted5['Afternoon']/len(data))*100,'Evening: ',(Counted5['Evening']/len(data))*100,'Night: ',
      (Counted5['Night']/len(data))*100)

# Outage Duration
print('Outage Duration (h)')
print('-------------------')
Counted = Counter(data.Outage_Duration)
print('1 < d <= 2: ',(Counted['1<Otg<=2']/len(data))*100,'d > 2: ',
      (Counted['Otg>2']/len(data))*100,'d <= 1: ',(Counted['Otg<=1']/len(data))*100)

# Switch Gear
print('Switch Gear')
print('-----------')
Counted = Counter(data.Switch_Gear)
print(Counted)
print('H7: ',(Counted['H7']/len(data))*100,'H3: ',(Counted['H3']/len(data))*100,'H10: ',
      (Counted['H10']/len(data))*100)

# Volt Level
print('Volt Level')
print('----------')
Counted = Counter(data.Volt_Level)
print('0.2 <= u <= 1.0: ',(Counted['0.2 <= u <= 1.0']/len(data))*100,'10 <= u <= 12: ',
      (Counted['10 <= u <= 12']/len(data))*100)

# Cause
print('Cause')
print('-----')
Counted = Counter(data.Cause)
print('Fabrication Fault: ',(Counted['Fabrication fault']/len(data))*100,
      'Fuse break: ',(Counted['Fuse break']/len(data))*100,
      'Digging: ',(Counted['Digging']/len(data))*100,
      'Overload: ',(Counted['Overload']/len(data))*100,
      'Incorrect assembling/installation: ',(Counted['Incorrect assembling/installation']/len(data))*100,
      'Unknown: ',(Counted['Unknown']/len(data))*100,
      'Traffic: ',(Counted['Traffic']/len(data))*100,
      'Incorrect operating: ',(Counted['Incorrect operating']/len(data))*100,
      'Lack of maintenance: ',(Counted['Lack of maintenance']/len(data))*100,
      'Thunder: ',(Counted['Thunder']/len(data))*100,
      'Wind: ',(Counted['Wind']/len(data))*100,
      'Incorrect method/instruction: ',(Counted['Incorrect method/instruction']/len(data))*100,
      'Other: ',(Counted['Other']/len(data))*100,
      'Subotage: ',(Counted['Subotage']/len(data))*100, 
            'Falling tree by wind: ',(Counted['Falling tree by wind']/len(data))*100, 
      'Rain: ',(Counted['Rain']/len(data))*100)

# Facility Part
print('Facility Part')
print('-------------')

Counted = Counter(data.Facility_Part)
print('Ground Cable Pillar: ',(Counted['Ground cable pillar']/len(data))*100,
      'Ground Feeder Cable in Ground: ',(Counted['Ground feeder cable in ground']/len(data))*100,
      'Ground Cable Fuse / Apparatus Box: ',(Counted['Ground cable fuse-/apparatus box']/len(data))*100,
      'Concr. Sec. Sub. Indoor Man.: ',(Counted['Concr.sec.substation indoor man.']/len(data))*100,
      'OH Line: ',(Counted['OH line']/len(data))*100,
      'Lodged Sec. Sub.: ',(Counted['Lodged secondary substation']/len(data))*100,
      'Metal Shielded Sec. Sub.: ',(Counted['Metal shielded sec. subst.']/len(data))*100,
      'Concr. Sec. Sub. Outdoor Man.: ',(Counted['Concr.sec.substation outdoor man.']/len(data))*100,
      'Pole Mounted Sec. Sub.: ',(Counted['Pole mounted sec. subst.']/len(data))*100,
      'Primary Sub.: ',(Counted['Primary substation']/len(data))*100,
      'Other Sec. Sub.: ',(Counted['Other secondary substation']/len(data))*100,
      'GFC in Water: ',(Counted['Ground feeder cable in water']/len(data))*100)

Year
------
After 2011:  57.9963789981895 Before 2011:  42.0036210018105
Season
------
Autumn:  22.57091128545564 Winter:  23.71756185878093 Spring:  25.890162945081475 Summer:  27.821363910681956
Hour
----
Morning:  39.52926976463488 Afternoon:  36.14966807483403 Evening:  21.24321062160531 Night:  3.0778515389257697
Outage Duration (h)
-------------------
1 < d <= 2:  41.52082076041038 d > 2:  31.985515992757996 d <= 1:  26.493663246831623
Switch Gear
-----------
Counter({'H7': 688, 'H3': 431, 'H10': 253, 'H4': 229, 'H2': 39, 'LINEHED EON': 13, 'H8': 4})
H7:  41.52082076041038 H3:  26.0108630054315 H10:  15.268557634278817
Volt Level
----------
0.2 <= u <= 1.0:  92.27519613759807 10 <= u <= 12:  7.724803862401932
Cause
-----
Fabrication Fault:  30.235365117682562 Fuse break:  25.165962582981294 Digging:  15.811707905853952 Overload:  9.71635485817743 Incorrect assembling/installation:  5.853952926976463 Unknown:  3.862401931200966 Traffic:  1.6294508147254072 Incorrect operating:  1.

Now that you are familiar with the data, we proceed to the main part of the lab which is the implementaiton of the bayesian networks.

### Environment setup

For the rest of the lab, we need to work the BAYESIAN NETWORK SOFTWARE FOR ARTIFICIAL INTELLIGENCE, Bayes Server. To download the software you need to go to the following webpage:

https://www.bayesserver.com/

In the Download page, you need to fill your personal information and then click on the “Email download details”. After a few minutes, the download link will be sent to your email. Click on the link and install the software.

For this lab, we are using the trial mode of the software. Therefore, when you open the Bayes Server software, select the 'trial' version of the software. Then click OK.

Note that in the trial version you cannot save the created networks. Moreover, every session is limited to 120 minutes. In the learning center (https://www.bayesserver.com/docs/), you can find help, tutorial, and videos about how to use the software.

In this lab, we are mainly working with structure learning and parameter learning:
- https://www.bayesserver.com/docs/learning/structural-learning
- https://www.bayesserver.com/docs/learning/parameter-learning


### Task 1: data_artificial.xlsx

a) Load the nodes from data_artificial.xlsx dataset by using the feature “Nodes from data”. Then, add some connections between the nodes by using the feature “Link”. Now you have the structure of your BN. Use the feature “Parameter learning” to learn the probability distributions of your BN based on the connections you have created. Finally, click on at the bottom of the page that you have created your network to see the parameters. In your report, you need to show your BN and the corresponding parameters. You can save your BN and the parameters using File >>> Save as image tab.

b) In the parameter learning, the performance of your BN based on the available data is represented by Log Likelihood, and BIC as it is shown in the following figure.

<img src="Log Liklihood.png" width = "600">
<br> Figure 1 The summary of a candidate network<br>
<br>Log Likelihood - the log likelihood of the data, given the candidate network.<br>
BIC - Bayesian Information Criterion

In your report, explain what each of these measures represent.

c) Construct two more networks (the same way as task 1.a). Compare the results of BIC and Log Likelihood of these three networks together. In your report, you need to show the networks and corresponding parameters. Also you need to write the BIC and Log Likelihood of these networks. Furthermore, you need to explain your conclusions about the three networks considering the performance measures.

d) Start a new network and load the nodes from data_artificial.xlsx dataset by using the feature “Nodes from data”. Then use the feature “Structure learning” to automatically learn the links between different attributes in the data file. Use the feature “Parameter learning” to learn the probability distributions of the BN. In your report, you need to show the networks and corresponding parameters. Also, explain the results of the performance measures in comparison to other networks that you have created in the previous steps.

### Task 2: data_real.xlsx<br>

a) Repeat all the requirements of task 1 using the dataset data_real.xlsx. In your report, you need to show all the four networks (for the first three networks you define the links, and for the fourth network the links are learnt using “Structure learning”) with the corresponding parameters and performance measures. You also need to explain the results.
 
b) Find the following conditional probabilities for the network which the links are learnt using the  “Structure learning”. Write the results in your report.

1. 𝑝(𝐶𝑎𝑢𝑠𝑒 = 𝐷𝑖𝑔𝑔𝑖𝑛𝑔 | 𝑆𝑒𝑎𝑠𝑜𝑛 = 𝑆𝑝𝑟𝑖𝑛𝑔) =?
2. 𝑝(𝑆𝑒𝑎𝑠𝑜𝑛 = 𝑆𝑝𝑟𝑖𝑛𝑔 | 𝐶𝑎𝑢𝑠𝑒 = 𝐷𝑖𝑔𝑔𝑖𝑛𝑔) =?
3. 𝑝(𝑆𝑒𝑎𝑠𝑜𝑛 = 𝐴𝑢𝑡𝑢𝑚𝑛 | 𝐶𝑎𝑢𝑠𝑒 = 𝐹𝑎𝑙𝑙𝑖𝑛𝑔 𝑡𝑟𝑒𝑒 𝑏𝑦 𝑤𝑖𝑛𝑑) =?
4. 𝑝(𝑂𝑢𝑡𝑎𝑔𝑒 𝑑𝑢𝑟𝑎𝑡𝑖𝑜𝑛 = 𝑂𝑡𝑔 > 2 | 𝐹𝑎𝑐𝑖𝑙𝑖𝑡𝑦 𝑝𝑎𝑟𝑡 = 𝐺𝑟𝑜𝑢𝑛𝑑 𝑐𝑎𝑏𝑙𝑒 𝑝𝑖𝑙𝑙𝑎𝑟) =?
5. 𝑝(𝐹𝑎𝑐𝑖𝑙𝑖𝑡𝑦 𝑝𝑎𝑟𝑡 = 𝐺𝑟𝑜𝑢𝑛𝑑 𝑐𝑎𝑏𝑙𝑒 𝑝𝑖𝑙𝑙𝑎𝑟|𝑆𝑤𝑖𝑡𝑐ℎ𝑔𝑒𝑎𝑟 = 𝐻3, 𝐶𝑎𝑢𝑠𝑒 = 𝐹𝑢𝑠𝑒 𝑏𝑟𝑒𝑎𝑘) =?
6. 𝑝(𝐹𝑎𝑐𝑖𝑙𝑖𝑡𝑦 𝑝𝑎𝑟𝑡 = 𝐺𝑟𝑜𝑢𝑛𝑑 𝑓𝑒𝑒𝑑𝑒𝑟 𝑐𝑎𝑏𝑙𝑒 𝑖𝑛 𝑔𝑟𝑜𝑢𝑛𝑑 | 𝐶𝑎𝑢𝑠𝑒 =
(𝐷𝑖𝑔𝑔𝑖𝑛𝑔, 𝐹𝑎𝑏𝑟𝑖𝑐𝑎𝑡𝑖𝑜𝑛 𝑓𝑎𝑢𝑙𝑡), 𝑆𝑤𝑖𝑡𝑐ℎ𝑔𝑒𝑎𝑟 = 𝐻3, 𝑆𝑒𝑎𝑠𝑜𝑛 = 𝑆𝑢𝑚𝑚𝑒𝑟) =?
7. 𝑝(𝐶𝑎𝑢𝑠𝑒 = 𝐷𝑖𝑔𝑔𝑖𝑛𝑔 | 𝐹𝑎𝑐𝑖𝑙𝑖𝑡𝑦 𝑝𝑎𝑟𝑡 = 𝑂𝐻 𝑙𝑖𝑛𝑒, 𝑆𝑤𝑖𝑡𝑐ℎ𝑔𝑒𝑎𝑟 = 𝐻3) =?
8. 𝑝(𝐹𝑎𝑐𝑖𝑙𝑖𝑡𝑦 𝑝𝑎𝑟𝑡 = 𝐺𝑟𝑜𝑢𝑛𝑑 𝑐𝑎𝑏𝑙𝑒 𝑝𝑖𝑙𝑙𝑎𝑟 | 𝑂𝑢𝑡𝑎𝑔𝑒 𝑑𝑢𝑟𝑎𝑡𝑖𝑜𝑛 = 𝑂𝑡𝑔 £ 1) =?


c) In your report, write 10 more of such examples with the corresponding probabilities. 

d) In your report, explain why when we are selecting an attribute of a node (changing the probability of the node to 100%), not only the probability values of its child node are changing but also the probability values of the parent nodes are changing. Support your explanations by using some examples and their corresponding probability equations.

e) Suppose you want to do diagnostic on the smart grid base on the constructed network and probability values. Explain in your report, which attributes would maximize the probability of failure in “Ground feeder cable in ground”? Which attributes would maximize the probability of having “Fabrication fault” as the cause of failure?

f )Which attributes are highly correlated with each other compared to other attributes? In your report, provide at least 6 examples and interpret the results.

Example:<br>
𝐶𝑎𝑢𝑠𝑒 = 𝑂𝑣𝑒𝑟𝑙𝑜𝑎𝑑 is highly correlated with 𝑆𝑒𝑎𝑠𝑜𝑛 = 𝑊𝑖𝑛𝑡𝑒𝑟 >>> 𝑝(𝐶𝑎𝑢𝑠𝑒 = 𝑂𝑣𝑒𝑟𝑙𝑜𝑎𝑑 |𝑆𝑒𝑎𝑠𝑜𝑛 = 𝑊𝑖𝑛𝑡𝑒𝑟 ) = 18.38 is the highest among all the other seasons. This means if we know that “it is Winter”, then the probability of having a failure caused by “Overload” is higher (more than two times higher) compared to other seasons.